In [ ]:
#Importing necessary libraries and classes
from pandas import read_csv, get_dummies, Series,DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from imblearn.over_sampling import SMOTE
df = read_csv('passenger satisfaction.csv')


In [ ]:
#Analyze dataset, its columns datatype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [ ]:
#Remove unnecessary columns
df.drop('id',axis=1,inplace = True)
df.drop('Unnamed: 0',axis=1,inplace = True)

In [ ]:
df['satisfaction'].unique()

array(['neutral or dissatisfied', 'satisfied'], dtype=object)

In [ ]:
#Mapping string type columns to numberic
df['Gender'] = df['Gender'].map({'Male':1, 'Female':0})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':1, 'Business travel':2})
df['Class'] = df['Class'].map({'Eco Plus':2, 'Business':3, 'Eco':1})
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':1, 'satisfied':0})

In [ ]:
df.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,1,13,1,2,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,1
1,1,0,25,2,3,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,1
2,0,1,26,2,3,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,0
3,0,1,25,2,3,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,1
4,1,1,61,2,3,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,0


In [ ]:
#Split data into Features and Labels
X = df.drop('satisfaction',axis =1)
Y = df['satisfaction']

In [ ]:
#Column X['Arrival Delay in Minutes'] has 'NaN' values, hence we have replace it with mean() value to that column
X['Arrival Delay in Minutes'] = X['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].mean())

In [ ]:
#Scaling the data
X_scaled = StandardScaler().fit_transform(X)
DataFrame(X_scaled)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.015031,0.472767,-1.745279,-1.490614,-0.029187,-0.731539,0.203579,0.616172,0.173776,-1.547323,...,1.183099,1.231704,0.479403,-0.266840,0.311769,0.549799,1.156436,1.305870,0.266393,0.073014
1,1.015031,-2.115208,-0.951360,0.670865,1.009393,-0.957184,0.203579,-0.695245,0.173776,0.018094,...,-1.849315,-1.769081,-1.849161,1.253380,-0.535045,-1.821012,0.305848,-1.742292,-0.361375,-0.237539
2,-0.985192,0.472767,-0.885200,0.670865,1.009393,-0.047584,-0.549533,-0.695245,-0.541060,-0.764614,...,1.183099,1.231704,0.479403,-0.266840,0.311769,0.549799,0.305848,1.305870,-0.387532,-0.392816
3,-0.985192,0.472767,-0.951360,0.670865,1.009393,-0.629246,-0.549533,1.271880,1.603448,1.583511,...,-1.091211,-1.018885,-1.072973,1.253380,-0.535045,-1.821012,0.305848,-0.980251,-0.099805,-0.159901
4,1.015031,0.472767,1.430397,0.670865,1.009393,-0.978244,0.203579,-0.039537,0.173776,0.018094,...,1.183099,-0.268688,-0.296785,0.493270,0.311769,-0.240472,-0.544740,-0.218211,-0.387532,-0.392816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,-0.985192,-2.115208,-1.083680,0.670865,-1.067767,-1.000307,-0.549533,-1.350954,-0.541060,0.018094,...,-1.091211,-1.018885,-0.296785,-1.787061,0.311769,-1.030742,-0.544740,-0.980251,-0.309061,-0.392816
103900,1.015031,0.472767,0.636478,0.670865,1.009393,1.160869,0.956691,0.616172,0.888612,0.800803,...,1.183099,1.231704,1.255590,1.253380,1.158582,1.340069,1.156436,0.543829,-0.387532,-0.392816
103901,1.015031,-2.115208,-0.620561,0.670865,1.009393,0.807860,-1.302646,-1.350954,-1.255895,0.018094,...,1.183099,0.481508,-0.296785,-1.026951,0.311769,1.340069,1.156436,0.543829,-0.204433,-0.030504
103902,-0.985192,-2.115208,-1.149840,0.670865,-1.067767,-0.189991,-1.302646,-1.350954,-1.255895,1.583511,...,-1.849315,-1.769081,0.479403,1.253380,-2.228672,1.340069,0.305848,-1.742292,-0.387532,-0.392816


In [ ]:
#Support Vector Classifier

from sklearn import svm

#Creating the pipeline with the support vector classifier
SVM_classifier2 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', svm.SVC(random_state=10) ) ])

#Defining the parameter for support vector classifier
kernels_c = {'classification__kernel': ['rbf'], 'classification__C': [10]}
grid_search = GridSearchCV(estimator=SVM_classifier2, param_grid=kernels_c, scoring='recall', cv=5)

#Fitting data to grid search cv
grid_search.fit(X_scaled, Y)
best_parameters = grid_search.best_params_
print(best_parameters)
best_result = grid_search.best_score_
print(best_result)


{'classification__C': 10, 'classification__kernel': 'rbf'}
0.967509626257731


In [ ]:
# Logistic Regression Classifier

from sklearn.linear_model import SGDClassifier

#Creating the pipeline with the logistic regression classifier
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),   # Synthetic Minority Oversampling Technique
        ('classification', SGDClassifier(loss = 'log_loss', penalty = 'elasticnet', random_state = 1))
    ])

#Defining the parameter grid for the logistic regression
grid_param = {'classification__eta0': [.001], 'classification__max_iter' : [100], 'classification__alpha': [.001], 'classification__l1_ratio': [1]}
gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='recall', cv=5, error_score = 'raise')

#Fitting data to grid search cv
gd_sr.fit(X_scaled, Y)
best_result = gd_sr.best_score_
print("Best result: ", best_result)


Best result:  0.8762547164912767


In [ ]:
# Ada boost classifier
from sklearn.ensemble import  AdaBoostClassifier
import pandas as pd

#Creating the pipeline with the Ada boost classifier
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', AdaBoostClassifier(random_state=1))
    ])

#Defining the parameter grid for the Ada boost
grid_param = {'classification__n_estimators': [40]}
gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='recall', cv=5)

#Fitting data to grid search cv
gd_sr.fit(X_scaled, Y)
best_result = gd_sr.best_score_
print(best_result)



0.9326245297932243


In [ ]:
#Random forest classifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble

#Creating the pipeline with random forest classifier
RF_classifier1 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', ensemble.RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=1))])

#Defining the parameter grid for random forest classifier
no_trees = {'classification__n_estimators': [300]}

grid_search2 = GridSearchCV(estimator=RF_classifier1, param_grid=no_trees, scoring='recall', cv=5)

#Fitting data to grid search cv
grid_search2.fit(X_scaled, Y)
best_result = grid_search2.best_score_
print(best_result)

0.9780736912097294


In [ ]:
# KNN classifier
from sklearn.neighbors import KNeighborsClassifier

# Applying SMOTE separately
X_resampled, Y_resampled = SMOTE(random_state=101).fit_resample(X_scaled, Y)
knn_classifier = KNeighborsClassifier()

#Creating the pipeline with the K-NN classifier
model = Pipeline([
    ('classification', knn_classifier)
])

#Defining the parameter grid for the K-NN classifier
grid_param = {'classification__n_neighbors': [3],
              'classification__algorithm': ['auto']}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='recall', cv=5)

#Fitting data to grid search cv
gd_sr.fit(X_resampled, Y_resampled)
best_result = gd_sr.best_score_
print(best_result)


0.9412015629327056


Calculating Precision


In [ ]:
# KNN classifier Precision
from sklearn.neighbors import KNeighborsClassifier

# Applying SMOTE separately
X_resampled, Y_resampled = SMOTE(random_state=101).fit_resample(X_scaled, Y)
knn_classifier = KNeighborsClassifier()

#Creating the pipeline with the K-NN classifier
model = Pipeline([
    ('classification', knn_classifier)
])

#Defining the parameter grid for the K-NN classifier
grid_param = {'classification__n_neighbors': [3],
              'classification__algorithm': ['auto']}

gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

#Fitting data to grid search cv
gd_sr.fit(X_resampled, Y_resampled)
best_result = gd_sr.best_score_
print(best_result)


0.929648321568647


In [ ]:
# Ada boost classifier
from sklearn.ensemble import  AdaBoostClassifier
import pandas as pd

#Creating the pipeline with the Ada boost classifier
model = Pipeline([
        ('balancing', SMOTE(random_state = 101)),
        ('classification', AdaBoostClassifier(random_state=1))
    ])

#Defining the parameter grid for the Ada boost
grid_param = {'classification__n_estimators': [40]}
gd_sr = GridSearchCV(estimator=model, param_grid=grid_param, scoring='precision', cv=5)

#Fitting data to grid search cv
gd_sr.fit(X_scaled, Y)
best_result = gd_sr.best_score_
print(best_result)



0.9352339857548826


In [17]:
#Random forest classifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble

#Creating the pipeline with random forest classifier
RF_classifier1 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', ensemble.RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=1))])

#Defining the parameter grid for random forest classifier
no_trees = {'classification__n_estimators': [300]}

grid_search2 = GridSearchCV(estimator=RF_classifier1, param_grid=no_trees, scoring='precision', cv=5)

#Fitting data to grid search cv
grid_search2.fit(X_scaled, Y)
best_result = grid_search2.best_score_
print(best_result)

0.9566141730896923
